In [14]:
from pathlib import Path

import tkinter as tk
from tkinter import filedialog
import json

import re

In [6]:
root = tk.Tk()
root.withdraw()

file_path = Path(filedialog.askopenfilename())

In [25]:
def get_map_dimensions(file_path:Path):
  """Gets the measured map's dimensions (in pixels) assuming that the filename contains this information

  Args:
      file_path (Path): pathlib.Path object of the considered map's data
  """
  map_dimensions = re.findall(
      '[0-9]{3}x[0-9]{3}',
      file_path.name
  )[0].split('x')

  return([int(x) for x in map_dimensions])

def get_metadata(file_path:Path):
  """Load metadata from the metadata file corresponding to the selected data file

  Args:
      file_path (Path): pathlib.Path object of the considered map's data
  """
  metadata_path = file_path.with_suffix('.libsmetadata')
  if metadata_path.is_file():
    with open(
      file_path.with_suffix('.libsmetadata'),'r'
    ) as file:

      return(json.load(file))
  else:
    raise ImportError('Metadata file is missing')

In [26]:
get_metadata(file_path)

ImportError: metadata file is missing

In [ ]:
if file_path.is_file():    
    print(f"Importing data from {file_path}")
    with open(file_path, "r") as f:        
        file_contents = f.read()
        print(f"The contents of the file are:\n{file_contents}")
else:
    # If the file does not exist, inform the user and prompt them to try again with a different file path
    print(f"File not found at {file_path}. Please try again with a different file path.")

In [ ]:
file_path = input("Please enter the path of your file: ")

if os.path.isfile(file_path):
    # If the file exists, we can proceed with reading its contents or performing other operations
    print(f"File found at {file_path}")
    with open(file_path, "r") as f:
        # Perform operations on the file here, e.g. reading its contents
        file_contents = f.read()
        print(f"The contents of the file are:\n{file_contents}")
else:
    # If the file does not exist, inform the user and prompt them to try again with a different file path
    print(f"File not found at {file_path}. Please try again with a different file path.")